# Tutorial 3 : Text classification using scikit learn

In [6]:
#Loading the data set - training data.
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

# Question 1 : Print the categories names and some text examples

In [47]:
# You can check the target names (categories) by using target_names
# the texts are stored in the data attribute of the variable twinty train
print(twenty_train.target_names)
print(twenty_train.data[56])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: eliot@lanmola.engr.washington.edu (eliot)
Subject: Re: Improvements in Automatic Transmissions
Keywords: Saturn, Subaru, manual, automatic
Article-I.D.: engr.Apr19.045221.19525
Organization: clearer than blir
Lines: 40
NNTP-Posting-Host: lanmola.engr.washington.edu


an excellent automatic can be found in the subaru legacy.  it switches to
"sport" mode when the electronics figure it,  not when the driver sets
the switch.. which is the proper way to do it, IMO.  so what does "sport"
mode entail?  several things:

1) revving to red line (or to the rev limiter in the case of the legacy)

2) delay

# Features extraction from text data

In [8]:
#check the scikit documentation to see definition of CoutVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape
#feature_name = count_vect.get_feature_names()
#print(feature_name)

(11314, 130107)

In [9]:
# TF-IDF is an other text preprocessing procedure : Check its definition and apply it using the following code TF-IDF处理
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(11314, 130107)


# Try a machine learning algorithm

In [10]:
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
print("分类器的相关信息：")
print(clf)

分类器的相关信息：
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


# Build a pipeline including preprocessings and learning steps

In [11]:
# Building a pipeline: We can write less code and do all of the above, by building a pipeline as follows:
# We will be using the 'text_clf' going forward.
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

# Evaluate your model performance

Exercice : evaluate the performance of the first model (named clf) on the 20 newsgroups test data 

In [12]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(twenty_test.data)
# 计算预测结果的准确率
print("朴素贝叶斯准确率为：")
print(np.mean(predicted ==twenty_test.target))
#print(test_data.target)
#print(docs_test)

朴素贝叶斯准确率为：
0.7738980350504514


# Exercice : Using scikit pipeline and SVM Classifier (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html), create a model based on the 20newsgroup dataset and evaluate its performance.

In [13]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge',
                                            penalty='l2',
                                            alpha=1e-3,
                                            n_iter=5,
                                            random_state=42)),
                    ])
_ = text_clf.fit(twenty_train.data, twenty_train.target)
predicted_SVM = text_clf.predict(twenty_test.data)
print("SVM准确率：")
print(np.mean(predicted_SVM == twenty_test.target))

from sklearn import metrics
print("打印分类性能指标：")
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))
print("打印混淆矩阵：")
metrics.confusion_matrix(twenty_test.target, predicted)

/Users/haosu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SVM准确率：
0.8238183749336165
打印分类性能指标：
                          precision    recall  f1-score   support

             alt.atheism       0.80      0.52      0.63       319
           comp.graphics       0.81      0.65      0.72       389
 comp.os.ms-windows.misc       0.82      0.65      0.73       394
comp.sys.ibm.pc.hardware       0.67      0.78      0.72       392
   comp.sys.mac.hardware       0.86      0.77      0.81       385
          comp.windows.x       0.89      0.75      0.82       395
            misc.forsale       0.93      0.69      0.80       390
               rec.autos       0.85      0.92      0.88       396
         rec.motorcycles       0.94      0.93      0.93       398
      rec.sport.baseball       0.92      0.90      0.91       397
        rec.sport.hockey       0.89      0.97      0.93       399
               sci.crypt       0.59      0.97      0.74       396
         sci.electronics       0.84      0.60      0.70       393
                 sci.med       0.92   

array([[166,   0,   0,   1,   0,   1,   0,   0,   1,   1,   1,   3,   0,
          6,   3, 123,   4,   8,   0,   1],
       [  1, 252,  15,  12,   9,  18,   1,   2,   1,   5,   2,  41,   4,
          0,   6,  15,   4,   1,   0,   0],
       [  0,  14, 258,  45,   3,   9,   0,   2,   1,   3,   2,  25,   1,
          0,   6,  23,   2,   0,   0,   0],
       [  0,   5,  11, 305,  17,   1,   3,   6,   1,   0,   2,  19,  13,
          0,   5,   3,   1,   0,   0,   0],
       [  0,   3,   8,  23, 298,   0,   3,   8,   1,   3,   1,  16,   8,
          0,   2,   8,   3,   0,   0,   0],
       [  1,  21,  17,  13,   2, 298,   1,   0,   1,   1,   0,  23,   0,
          1,   4,  10,   2,   0,   0,   0],
       [  0,   1,   3,  31,  12,   1, 271,  19,   4,   4,   6,   5,  12,
          6,   3,   9,   3,   0,   0,   0],
       [  0,   1,   0,   3,   0,   0,   4, 364,   3,   2,   2,   4,   1,
          1,   3,   3,   4,   0,   1,   0],
       [  0,   0,   0,   1,   0,   0,   2,  10, 371,   0,   0,  

# Grid Search : Trying different hyperparameters

In [14]:
# First, we are creating a list of parameters for which we would like to do performance tuning. 
# All the parameters name start with the classifier name.
# E.g. vect__ngram_range; here we are telling to use unigram and bigrams and choose the one which is optimal.

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [15]:
# Next, we create an instance of the grid search by passing the classifier, parameters 
# and n_jobs=-1 which tells to use multiple cores from user machine.

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

/Users/haosu/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/haosu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [69]:
# To see the best mean score and the params, run the following code

print(gs_clf.best_score_)
gs_clf.best_params_
# 详细 gs_clf.cv_results_

# Output for above should be: The accuracy has now increased to ~90.6% for the NB classifier
# and the corresponding parameters are {‘clf__alpha’: 0.01, ‘tfidf__use_idf’: True, ‘vect__ngram_range’: (1, 2)}.

0.6025


{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

# Grid Search for SVM : Similarly, do grid search for SVM and output the best score and parameters

In [17]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])
print("最佳准确率：%r" % (gs_clf.best_score_))

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

/Users/haosu/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/haosu/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/haosu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:130: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


最佳准确率：0.6025
clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


# Stemming using NLTK

## Stemming is the process of reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words known as a lemma. Stemming help you improve your model performances

## Stop words removal is also an other technique used in text classification in order to improve the model performances : Take a look at this webpage for more information : https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

In [18]:
import nltk

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))
print(len(twenty_train.data))
dataBank = []

for article in twenty_train.data:
    dataBank.append(word_tokenize(article))


# print(word_tokenize(twenty_train.data[0]))

11314


In [19]:
# NLTK process
stop_words.add("I")
stop_words.add(",")
stop_words.add("@")
stop_words.add("(")
stop_words.add(")")
stop_words.add("-")
stop_words.add("_")
stop_words.add(".")
stop_words.add(":")
stop_words.add("A")
stop_words.add("<")
stop_words.add(">")
stop_words.add("n't")
stop_words.add("'s")
stop_words.add("?")
stop_words.add("!")
stop_words.add("--")
twenty_train_split = []
for i in range(len(dataBank)):
    filtered_sentence = []
    for word in dataBank[i]:
        if word not in stop_words:
            filtered_sentence.append(word)
    twenty_train_split.append(filtered_sentence)


In [68]:
for i in len(twenty_train_split):
    twenty_train_split[i] = " ".join( twenty_train_split[i])

TypeError: 'int' object is not iterable

In [64]:
print(" ".join(twenty_train_split))

TypeError: sequence item 0: expected str instance, list found

In [48]:
docs_new = ['God is love','stage']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)
for i in predicted:
    print(twenty_train.target_names[i])
#for doc,category in zip(docs_new,predicted):
    #print("%r => %s") %(doc,twenty_train.target_names[category])


soc.religion.christian
sci.space
